In [44]:
import pandas as pd

# Path to the Excel file
excel_file_path = "HINDALCO_1D.xlsx"

# Read the Excel file into a DataFrame
df = pd.read_excel(excel_file_path)

print(df)

       datetime   close    high     low    open   volume instrument
0    2014-01-24  114.00  115.35  113.00  113.15  5737135   HINDALCO
1    2014-01-27  111.10  112.70  109.30  112.00  8724577   HINDALCO
2    2014-01-28  113.80  115.00  109.75  110.00  4513345   HINDALCO
3    2014-01-29  111.75  114.75  111.15  114.50  4713458   HINDALCO
4    2014-01-30  108.10  110.70  107.60  110.20  5077231   HINDALCO
...         ...     ...     ...     ...     ...      ...        ...
1210 2018-12-21  224.00  227.45  221.80  224.25  6933691   HINDALCO
1211 2018-12-24  217.85  224.00  216.95  224.00  4667022   HINDALCO
1212 2018-12-26  218.60  219.80  212.90  213.85  6554865   HINDALCO
1213 2018-12-27  220.60  223.90  219.60  221.40  7947144   HINDALCO
1214 2018-12-28  223.50  226.05  221.00  221.00  5074920   HINDALCO

[1215 rows x 7 columns]


In [45]:
import pymysql

# Connect to the MySQL server
cnx = pymysql.connect(
    host='localhost',
    user='root',
    password='',
)

# Create a cursor object to execute SQL queries
cursor = cnx.cursor()

In [46]:
def createDatabase(database_name):
    
    # SQL query to create the database
    create_database_query = f'CREATE DATABASE {database_name}'

    # Execute the query to create the database
    cursor.execute(create_database_query)
    cursor.close()
    cnx.close()

    # Connect to the newly created database
    

In [47]:
def connectToDatabase(database_name):
    global cnx
    global cursor
    cnx = pymysql.connect(
        host='localhost',
        user='root',
        password='',
        database=database_name
    )
    cursor = cnx.cursor()

In [48]:
def createTable(table_name):
    create_table_query = '''
    CREATE TABLE %s (
    datetime DATETIME,
    close DECIMAL(10, 2),
    open DECIMAL(10, 2),
    high DECIMAL(10, 2),
    low DECIMAL(10, 2),
    volume INT,
    instrument VARCHAR(255)
    );
    
    ''' % (table_name)

    # Execute the query to create the table
    cursor.execute(create_table_query)

    # Commit the changes to the database
    cnx.commit()
    #print(create_table_query)

In [49]:
def checkIfDatabaseExists(database_name):


    # SQL query to fetch the list of databases
    show_databases_query = 'SHOW DATABASES'

    # Execute the query
    cursor.execute(show_databases_query)

    # Fetch all the databases
    databases = cursor.fetchall()

    # Check if the database exists
    if (database_name,) in databases:
        return True
    else:
        return False


In [50]:
def checkIfTableExists(table_name):


    # SQL query to check if the table exists
    check_table_query = f"SHOW TABLES LIKE '{table_name}'"

    # Execute the query
    cursor.execute(check_table_query)

    # Fetch the result
    result = cursor.fetchone()

    # Check if the table exists
    if result:
        return True
    else:
        return False




In [51]:
if(checkIfDatabaseExists("invsto")):
    print("Database already exists")
else:
    createDatabase("invsto")

Database already exists


In [52]:
connectToDatabase("invsto")

In [53]:
# Close the cursor and connection
if(checkIfTableExists("invsto_hindalco_details")):
    print("Table already exists")
else:
    createTable("invsto_hindalco_details")


Table already exists


In [54]:
def insertValuesInTable(datetime,close,open_price,high,low,volume,instrument):

    sql = """INSERT INTO invsto_hindalco_details (datetime, close, open, high, low, volume, instrument)
    VALUES (%s, %f, %f, %f, %f, %d, %s)"""

    # Insert the values into the table
    sql = "INSERT INTO invsto_hindalco_details (datetime, close, open, high, low, volume, instrument) VALUES (%s, %s, %s, %s, %s, %s, %s)"
    cursor.execute(sql, (datetime, close, open_price, high, low, volume, 'HINDALCO'))

    # Commit the changes to the database
    cnx.commit()

In [55]:
def addDftoTable():
    
    for i in range(0,len(df.index)):

        row_wise_datetime = df.loc[i]["datetime"]
        row_wise_close = df.loc[i]["close"]
        row_wise_high = df.loc[i]["high"]
        row_wise_low = df.loc[i]["low"]
        row_wise_open = df.loc[i]["open"]
        row_wise_volume = df.loc[i]["volume"]
        row_wise_instrument = df.loc[i]["instrument"]
        insertValuesInTable(row_wise_datetime,row_wise_close,row_wise_open,row_wise_high,row_wise_low,row_wise_volume,row_wise_instrument)


    
    


In [56]:
#addDftoTable()

In [57]:
cursor.close()
cnx.close()